In [1]:
!pwd

/home/user/Downloads/Robotic/T5.2/v2


In [1]:
# %cd /content/
# !rm -rf yolact
!git clone https://github.com/dbolya/yolact.git
# !mkdir weights
# !wget https://download.pytorch.org/models/resnet50-19c8e357.pth -O weights/resnet50-19c8e357.pth

Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936 (from 1)
Receiving objects: 100% (2936/2936), 21.20 MiB | 18.80 MiB/s, done.
Resolving deltas: 100% (2002/2002), done.


In [1]:
%cd yolact_2025

/home/user/Downloads/Robotic/T5.2/v2/yolact_2025


In [2]:
# %%
# ========================================================================================
# CELL 1: UNIFIED SETUP, CLONE, PATCH, AND RESTART
# ========================================================================================
# Run this cell once at the very beginning of your session.

import os

# # --- 1. MOUNT DRIVE AND INSTALL DEPENDENCIES ---
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)
# !pip install pycocotools

# # --- 2. START WITH A FRESH CLONE ---
# print("Ensuring a clean start by re-cloning the repository...")
# %cd /content/
# !rm -rf yolact
# !git clone https://github.com/dbolya/yolact.git
# %cd /content/yolact/
# !mkdir -p weights
# !wget https://download.pytorch.org/models/resnet50-19c8e357.pth -O weights/resnet50-19c8e357.pth
# print("Fresh clone and weight download complete.")


# --- 3. APPLY ALL PATCHES TO THE FILES ON DISK ---
print("\nApplying all patches to files on disk...")

# PATCH 3A: Comprehensive fix for `utils/augmentations.py`
file_to_patch_aug = 'utils/augmentations.py'
try:
    with open(file_to_patch_aug, 'r') as f: content = f.read()
    if 'from numpy import random' in content:
        content = content.replace('from numpy import random', 'import random')
        content = content.replace('random.randint(2)', 'random.randint(0, 1)')
        content = content.replace('k = random.randint(4)', 'k = random.randint(0, 3)')
        content = content.replace('left = random.uniform(width - w)', 'left = random.uniform(0, width - w)')
        content = content.replace('top = random.uniform(height - h)', 'top = random.uniform(0, height - h)')
        with open(file_to_patch_aug, 'w') as f: f.write(content)
        print("-> Comprehensive patch applied for `random` library conflicts.")
    else: print("-> File utils/augmentations.py already appears to be patched.")
except FileNotFoundError: print(f"Error: Could not find the file {file_to_patch_aug}")

# PATCH 3B: Text Patch for `backbone.py`
file_to_patch_backbone = 'backbone.py'
original_line_backbone = "        self.load_state_dict(torch.load(path), strict=False)"
final_correct_line_backbone = "        device = next(self.parameters()).device\n        self.load_state_dict(torch.load(path, map_location=device, weights_only=False), strict=False)"
try:
    with open(file_to_patch_backbone, 'r') as f: content = f.read()
    if original_line_backbone in content:
        content = content.replace(original_line_backbone, final_correct_line_backbone)
        with open(file_to_patch_backbone, 'w') as f: f.write(content)
        print("-> Patch applied for ResNetBackbone weight loading.")
    else: print("-> File backbone.py already appears to be patched.")
except FileNotFoundError: print(f"Error: Could not find the file {file_to_patch_backbone}")

# PATCH 3C: Text Patch for `yolact.py` (THE CORRECTED TARGET)
file_to_patch_yolact = 'yolact.py'
bad_line_tensor = '                self.priors = torch.Tensor(prior_data, device=device).view(-1, 4).detach()'
good_line_tensor = '                self.priors = torch.tensor(prior_data, device=device, dtype=torch.float32).view(-1, 4).detach()'
try:
    with open(file_to_patch_yolact, 'r') as f: content = f.read()
    if bad_line_tensor in content:
        # Also bypass the cython nms import in this file
        content = content.replace("from utils.cython_nms import nms as cnms", "# from utils.cython_nms import nms as cnms # Patched out")
        content = content.replace(bad_line_tensor, good_line_tensor)
        with open(file_to_patch_yolact, 'w') as f: f.write(content)
        print("-> Patches applied to yolact.py for prior generation.")
    else: print("-> File yolact.py already appears to be patched.")
except FileNotFoundError: print(f"Error: Could not find the file {file_to_patch_yolact}")

print("\nAll setup and patching is complete. A kernel restart is required.")



Applying all patches to files on disk...
-> Comprehensive patch applied for `random` library conflicts.
-> Patch applied for ResNetBackbone weight loading.
-> Patches applied to yolact.py for prior generation.

All setup and patching is complete. A kernel restart is required.
